### kernel density estimation and kernel regression for prediction
This notebook is my first attempt to write down the ddiff and dddiff models that I have been working on

mixed recursive/iterative approach (rather than vectorized approach that is harder to generalize
- create separate submodules for KDE and KDreg
    - develop a system for storing differenced but unweighted data and the appropriate mask to lessen memory usage. 
        -1's or 0's depending on i/j/k/l/.... values
    - figure out the deepest level requested by the user and levels that are less deep are just slices of the deeper model. 
        -scout out the tree to build the differenced dataset. 
          
        
 plot and compare
on synthetic data, 1, 2, 3+ mixed distributions
1d - Ndiff vs gaussian kernel vs kernel_tunneling
2d -  


multidimensional x problem
e.g., parameter treatment
"product kernel approach" vs l2 "el two" (radial basis?)distance 

#### generate a simple linear dataset y=xb+e
##### store validation data from same dataset too

### create modeldict
- 'Ndiff_type': refers to the mathematical form of Ndiff
  - 'product'-Ndiff1 multiplied by Ndiff2
  - 'recursive'-Ndiff1's bw is Ndiff 2
- max_bw_Ndiff: is the depth of Ndiffs applied in estimating the bandwidth.
- 'normalize_Ndiffwtsum':
  - 'across' means sum across own level of kernelized-Ndiffs and divide by that sum (CDF approach)
  - 'own_n' means for (n+k)diff where n+K=max_bw_Ndiff
- Ndiff_bw_kern:
  - rbfkern means use the radial basis function kernel
  - 'product' means use product kernel like as in liu and yang eq1. 
- 'regression_model':
  - 'NW' means use nadaraya-watson kernel regression
  - 'full_logit' means local logit with all variables entering linearly
  - 'rbf_logit' means local logit with 1 parameters: scaled l2 norm centered on zero 
 (globally or by i?). Is this a new idea?
- outer_x_bw_form: if x is separated into blocks of rvars that are combined within a block using rbf kernel, 
  - 'one_for_all' - use same hx bw for all blocks
  - 'one_per_block' - each block or rvars gets an hx
- ykern_grid
  - if int, then create int evnely spaced values from -3 to 3 (standard normal middle ~99%)
  - 'no' means use original data points (self is masked when predicting self)
- xkern_grid
  - much like ykern_grid, but used less frequently since x's are typically pre-specified
- product_kern_norm: when multiplying kernels across random variables, this parameter determines if each random variable has its kernels normalized before the product or not
  - 'self' means each random variable has its kernels divided by the sum of kernels across the nout axis (the number of possibile values y is averaged over, whose probabilities sum to 1)
  - 'own_n' means same as self, but divided by count of non-masked items in second to last, nout lenght dimension
  - 'none' means no normalization prior to taking products across rvar kernels
- hyper_param_form_dict is a nested dictionary
  - Ndiff_exponent is the exponent wrapped around typically a sum of kernels for each Ndiff level 
after level 1 (i.e., (i-j) or centered, obvious/conventional level.)
  - x_bandscale is the parameter specific to each variable (each x in X) used for prediction (y)
  - Ndiff_depth_bw is used as the kernel's bandwidth (h) at each level of Ndiff including at level 1 
  - outer_x_bw vanilla bandwidth for the rbf or product kernel
  - outer_y_bw vanilla bandwidth for y

In [1]:
import os
import kernelcompare as kc
#from importlib import reload
networkdir='o:/public/dpatton/kernel'
mydir=os.getcwd()
#mydir2=os.path.join(mydir,"old models without datagen")
test=kc.KernelCompare(directory=mydir)

In [2]:
Ndiff_type_variations=('modeldict:Ndiff_type',['recursive','product'])
max_bw_Ndiff_variations=('modeldict:max_bw_Ndiff',[2,3])
Ndiff_start_variations=('modeldict:Ndiff_start',[1,2])
ykern_grid_variations=('modeldict:ykern_grid',[33])
#product_kern_norm_variations=('modeldict:product_kern_norm',['self','own_n'])#include None too?
#normalize_Ndiffwtsum_variations=('modeldict:normalize_Ndiffwtsum',['own_n','across'])
optdict_variation_list=[Ndiff_type_variations,ykern_grid_variations,max_bw_Ndiff_variations,Ndiff_start_variations]  #,product_kern_norm_variations,normalize_Ndiffwtsum_variations]



In [3]:
#the default datagen_dict as of 11/25/2019
#datagen_dict={'batch_n':32,'batchcount':10, 'param_count':param_count,'seed':1, 'ftype':'linear', 'evar':1, 'source':'monte'}


batch_n_variations=('batch_n',[32])
batchcount_variations=('batchcount',[20])
ftype_variations=('ftype',['linear','quadratic'])
param_count_variations=('param_count',[1,2])
#need to add block capability to datagen so we can repeat optimization with new training data.

datagen_variation_list=[batch_n_variations,batchcount_variations,ftype_variations,param_count_variations]



In [4]:
testrun=test.prep_model_list(optdict_variation_list=optdict_variation_list,datagen_variation_list=datagen_variation_list,verbose=1)
'''print(f'len(testrun){len(testrun)}')
print(f'testrun[0]{testrun[0:2]}')
print('--------------')
print(f'testrun[-1]{testrun[-4:]}')'''

dict_combo_list has 4 variations to run
len(datagen_dict_list):4
dict_combo_list has 8 variations to run
dict_combo_list has 8 variations to run
dict_combo_list has 8 variations to run
dict_combo_list has 8 variations to run


"print(f'len(testrun){len(testrun)}')\nprint(f'testrun[0]{testrun[0:2]}')\nprint('--------------')\nprint(f'testrun[-1]{testrun[-4:]}')"

In [ ]:
import traceback
from random import shuffle
shuffle(testrun)
#a_rundict=testrun[100]#this produced the Ndiff_exponent error for recursive Ndiff
for idx in range(len(testrun)):
    print(f'~~~~~~~run number:{idx}`~~~~~~~')
    a_rundict=testrun[idx]
    print(f'a_rundict{a_rundict}')
    optimizedict=a_rundict['optimizedict']
    datagen_dict=a_rundict['datagen_dict']

    try:
        test.do_monte_opt(optimizedict,datagen_dict,force_start_params=0)
        test.open_condense_resave('model_save',verbose=0)
        test.merge_and_condense_saved_models(merge_directory=None,save_directory=None,condense=None,verbose=None)
    except:
        print('traceback for run',idx)
        print(traceback.format_exc())


~~~~~~~run number:0`~~~~~~~
a_rundict{'optimizedict': {'opt_settings_dict': {'method': 'Nelder-Mead', 'options': {'xatol': 0.5, 'fatol': 1, 'adaptive': True}, 'mse_threshold': 2, 'help_start': 1, 'partial_match': 1}, 'hyper_param_dict': {'Ndiff_exponent': array([0.3, 0.3]), 'x_bandscale': array([1.]), 'outer_x_bw': array([2.7]), 'outer_y_bw': array([2.2]), 'Ndiff_depth_bw': array([0.5, 0.5]), 'y_bandscale': array([1.5])}, 'modeldict': {'Ndiff_type': 'product', 'param_count': 1, 'Ndiff_start': 1, 'max_bw_Ndiff': 2, 'normalize_Ndiffwtsum': 'own_n', 'xkern_grid': 'no', 'ykern_grid': 33, 'outer_kern': 'gaussian', 'Ndiff_bw_kern': 'rbfkern', 'outer_x_bw_form': 'one_for_all', 'regression_model': 'NW', 'product_kern_norm': 'self', 'hyper_param_form_dict': {'Ndiff_exponent': 'free', 'x_bandscale': 'non-neg', 'Ndiff_depth_bw': 'non-neg', 'outer_x_bw': 'non-neg', 'outer_y_bw': 'non-neg', 'y_bandscale': 'non-neg'}}}, 'datagen_dict': {'validate_batchcount': 10, 'batch_n': 32, 'batchcount': 20, 'pa

In [ ]:
#test.open_condense_resave('condensed_model_save',verbose=1)


In [ ]:

#test.recursive_merge(networkdir,overwrite=0,verbose=1)



In [ ]:
'''import numpy as np



datagen_dict={'train_n':60,'n':200, 'param_count':2,'seed':1, 'ftype':'linear', 'evar':1}
linear_dict_to_add_list=[(f'datagen_dict:{key}',val) for key,val in datagen_dict.items()]
print(linear_dict_to_add_list)

test.add_dict('model_save_linear',linear_dict_to_add_list,overwrite=0,verbose=0)
test.print_model_save(filename="model_save_linear-add_dict")


datagen_dict={'train_n':60,'n':200, 'param_count':2,'seed':1, 'ftype':'quadratic', 'evar':1}
quadratic_dict_to_add_list=[(f'datagen_dict:{key}',val) for key,val in datagen_dict.items()]
print(quadratic_dict_to_add_list)

test.add_dict('model_save_quadratic',quadratic_dict_to_add_list,overwrite=0,verbose=0)
test.print_model_save(filename="model_save_quadratic-add_dict")'''

In [ ]:
#test.print_model_save(filename="condensed_model_save")


In [ ]:
'''
#print(os.getcwd())
#try:test.open_condense_resave('model_save',verbose=0)
#except:pass
try:test.open_condense_resave('condensed_model_save',verbose=0)
except:pass'''

In [ ]:
'''from os import getcwd
thisdir=getcwd()
test.merge_and_condense_saved_models(merge_directory=thisdir,save_directory=thisdir,condense=1,verbose=1)'''

#-----------------Calibrate/Optimize--------

In [ ]:
'''import myreg
x=test.train_x
y=test.train_y
xfull=np.concatenate([np.ones([x.shape[0],1]),x],axis=1)
just_linreg=myreg.reg(y,xfull)
just_linreg.myreg()
print('linreg_mse:',just_linreg.mse)'''

In [ ]:
from bokeh.io import show, output_notebook#,curdoc,save, output_file
from bokeh.plotting import figure
#from bokeh.models import ColumnDataSource, Range1d, BBoxTileSource
#from bokeh.layouts import row
output_notebook()

In [ ]:
#xgrid=np.linspace(1,n,n)
x=quad_misspec.train_x
y=quad_misspec.train_y


xgrid=x[:,0]
yhat=quad_misspec.optimize_obj.yhat_un_std
p=figure(title='y and yhat', plot_width=900, plot_height=500)
p.xaxis.axis_label = 'observations'
p.scatter(xgrid,y,size=10,color='blue',alpha=0.4,legend='y')
#p.line(xgrid,precip,color='blue',alpha=0.6,legend='precipitation')
p.scatter(xgrid,yhat,size=5,color='green',alpha=0.9,legend='yhat')
p.circle(xgrid,just_linreg.yhat,color='red',legend='just lin reg yhat')
p.line(xgrid,np.mean(y))
p.legend.location = "top_left"
#p.yaxis.visible=False
show(p)

In [ ]:
print(yhat)
print(y)

In [ ]:

ypredict=optimized_Ndiff_kernel.predict_tool(xvalidate,optimized_Ndiff_kernel.fixed_or_free_paramdict,modeldict1)
prediction_err=yvalidate-ypredict
prediction_mse=np.mean(prediction_err**2)

In [ ]:
np.ma.count_masked(yhat)

## class toolcompare(object):